In [2]:
!pip install wandb
import wandb
wandb.login()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from wandb.keras import WandbCallback

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 定义wandb超参数扫描的配置
sweep_config = {
    'method': 'bayes',  # 可以使用 'random', 'grid', 'bayes'
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'min': 0.0001,
            'max': 0.01
        },
        'gru_units1': {
            'values': [128, 256, 512]
        },
        'gru_units2': {
            'values': [64, 128, 256]
        },
        'dropout_rate': {
            'min': 0.2,
            'max': 0.5
        },
        'reg_rate': {
            'min': 0.001,
            'max': 0.01
        }
    }
}

# 初始化wandb扫描
sweep_id = wandb.sweep(sweep_config, project="gru_model_optimization")

data_folder = '/content/drive/My Drive/DL2470 Final Project Group/EEG data/'
X_train = np.load(data_folder + 'X_train.npy')
X_test = np.load(data_folder + 'X_test.npy')
y_train = np.load(data_folder + 'y_train.npy')
y_test = np.load(data_folder + 'y_test.npy')


Create sweep with ID: sahip1wf
Sweep URL: https://wandb.ai/wxdycq/gru_model_optimization/sweeps/sahip1wf


In [5]:
# 定义模型构建函数
def build_model(input_shape, learning_rate, gru_units1, gru_units2, dropout_rate, reg_rate):
    model = Sequential([
        GRU(gru_units1, input_shape=input_shape, return_sequences=True, kernel_regularizer=l2(reg_rate)),
        BatchNormalization(),
        GRU(gru_units2, return_sequences=False, kernel_regularizer=l2(reg_rate)),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 定义训练函数
def train():
    with wandb.init() as run:
        config = wandb.config
        model = build_model((20, 9), config.learning_rate, config.gru_units1, config.gru_units2, config.dropout_rate, config.reg_rate)
        model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[WandbCallback()])

# 运行wandb代理开始扫描
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 794i7dsc with config:
wandb: 	dropout_rate: 0.4284946920955428
wandb: 	gru_units1: 256
wandb: 	gru_units2: 128
wandb: 	learning_rate: 0.003746627686184571
wandb: 	reg_rate: 0.0029098042193543647
wandb: Currently logged in as: lmy2001 (wxdycq). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
 99/100 [============================>.] - ETA: 0s - loss: 0.9628 - accuracy: 0.6675

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240501_010529-794i7dsc/files/model-best)... Done. 0.0s


100/100 [==============================] - 28s 232ms/step - loss: 0.9625 - accuracy: 0.6676 - val_loss: 0.7799 - val_accuracy: 0.7024
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 0.6959 - accuracy: 0.7010

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240501_024951-0awcw0mb/files/model-best)... Done. 0.0s


100/100 [==============================] - 36s 309ms/step - loss: 0.8496 - accuracy: 0.6813 - val_loss: 0.7244 - val_accuracy: 0.7238
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 0.6980 - accuracy: 0.7102

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240501_024951-0awcw0mb/files/model-best)... Done. 0.0s


100/100 [==============================] - 28s 283ms/step - loss: 0.6980 - accuracy: 0.7102 - val_loss: 0.6938 - val_accuracy: 0.6904
Epoch 3/10
100/100 [==============================] - 19s 189ms/step - loss: 0.6584 - accuracy: 0.7176 - val_loss: 0.7041 - val_accuracy: 0.6986
Epoch 4/10
 89/100 [=========================>....] - ETA: 1s - loss: 0.6398 - accuracy: 0.7145

wandb: Ctrl + C detected. Stopping sweep.
